<< [Sudoku-11-Comparison](Sudoku-11-Comparison.ipynb) | [Index](README.md) | [Fin](README.md) >>

# Résolution de Sudoku avec Automates Symboliques (Z3)

Dans ce notebook, nous explorons une approche alternative pour résoudre des puzzles de Sudoku en utilisant des **automates symboliques** via Z3. Cette approche modélise le Sudoku comme un système de transitions d'états symboliques, offrant une perspective différente des approches CSP classiques.

## Contexte : Pourquoi Automates Symboliques?

Les automates symboliques représentent les états et les transitions de manière **compacte** en utilisant des formules logiques au lieu d'énumérer explicitement chaque état. Pour le Sudoku :

- **État symbolique** : Configuration partielle représentée par des contraintes logiques
- **Transition** : Assigner une valeur à une cellule (ajout d'une contrainte)
- **État final** : Grille complète satisfaisant toutes les contraintes
- **Condition d'acceptation** : Toutes les contraintes Sudoku satisfaites

**Références** :
- [Search-12-SymbolicAutomata](../Search/Foundations/Search-12-SymbolicAutomata.ipynb) - Théorie des automates symboliques
- [Sudoku-4-Z3](Sudoku-4-Z3.ipynb) - Z3 basics pour Sudoku
- [Sudoku-3-ORTools](Sudoku-3-ORTools.ipynb) - Approche CSP classique

## Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :

1. **Comprendre** - Comment modéliser un Sudoku comme automate symbolique
2. **Implémenter** - Un solver Sudoku avec automates symboliques Z3
3. **Comparer** - Cette approche avec la méthode CSP classique (OR-Tools)
4. **Étendre** - L'approche symbolique à d'autres problèmes de contraintes

### Prérequis

- [Sudoku-4-Z3.ipynb](Sudoku-4-Z3.ipynb) - Bases de Z3
- [Search-12-SymbolicAutomata.ipynb](../Search/Foundations/Search-12-SymbolicAutomata.ipynb) - Automates symboliques
- [Sudoku-3-ORTools.ipynb](Sudoku-3-ORTools.ipynb) - Approche CSP (recommandé)

### Durée estimée : 1h30

### Voir aussi

- [SymbolicAI-Linq2Z3](../SymbolicAI/) - Série complète sur l'IA symbolique avec Z3
- [Search Foundations](../Search/Foundations/) - Algorithmes de recherche

---

## 1. Introduction - Sudoku comme Problème d'Automate (15 min)

### 1.1 Pourquoi Automates Symboliques?

Les approches classiques de résolution de Sudoku utilisent différentes modélisations :

| Approche | Modélisation | Avantages | Inconvénients |
|----------|----------------|-----------|---------------|
| **CSP (OR-Tools)** | Variables + domaines + contraintes | Intuitif, propagation efficace | Exploitation d'états explicite |
| **Backtracking** | Arbre de recherche | Simple à implémenter | Exponentiel dans le pire cas |
| **Automate Symbolique** | États + transitions symboliques | Représentation compacte, vérification formelle | Moins intuitif |

L'approche par **automate symbolique** offre :

- **Représentation compacte** : Un ensemble infini d'états peut être représenté par une seule formule
- **Vérification formelle** : Z3 garantit que toutes les contraintes sont satisfaites
- **Généricité** : Le même cadre s'applique à de nombreux problèmes de contraintes

### 1.2 Modélisation du Sudoku comme Automate

Un automate symbolique pour le Sudoku se compose de :

1. **Alphabet** Σ = {1, 2, ..., 9} - Les valeurs possibles pour une cellule
2. **État symbolique** - Assignation partielle représentée par des formules Z3
3. **Transitions** - Ajout d'une contrainte (assigner une valeur à une cellule)
4. **États acceptants** - Configurations complètes et valides

Visuellement, l'espace d'états symbolique du Sudoku :

```
État initial      Transition                État final
(grille vide)    ------------------>    (grille remplie)
  x_ij ∈ {1..9}    "fixer x_00 = 1"     All(row/col/block)
  pour tout i,j                         satisfaites
```

## 2. Automate Symbolique Simple - Contrainte de Ligne (20 min)

Commençons par un automate simple qui vérifie la contrainte de ligne : **toutes les valeurs d'une ligne doivent être distinctes**.

### 2.1 Installation de Z3

In [1]:
# Installation de z3-solver si nécessaire
!pip install -q z3-solver

from z3 import *
from typing import Optional

print(f"Z3 version: {get_version()}")

Z3 version: (4, 15, 4, 0)



[notice] A new release of pip is available: 25.0.1 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### 2.2 Classe RowAutomaton

Cet automate symbolique modélise une seule ligne de Sudoku. Il utilise un solver Z3 pour représenter l'état symbolique de la ligne.

In [2]:
class RowAutomaton:
    """
    Automate symbolique pour la contrainte de ligne Sudoku.
    
    Cet automate vérifie que toutes les valeurs d'une ligne sont distinctes
    et dans le domaine {1, ..., 9}.
    """
    
    def __init__(self, row_idx: int):
        """
        Initialise l'automate pour une ligne spécifique.
        
        Args:
            row_idx: Index de la ligne (0-8)
        """
        self.row_idx = row_idx
        self.ctx = Context()  # Contexte Z3 pour cet automate
        self.solver = SolverFor("QF_LIA", ctx=self.ctx)  # Solver pour arithmétique linéaire
        
        # Variables symboliques pour chaque cellule de la ligne
        # x_{row_idx}_{j} représente la valeur de la cellule (row_idx, j)
        self.cells = [
            Int(f'x_{row_idx}_{j}', ctx=self.ctx) 
            for j in range(9)
        ]
        
        # Contrainte de ligne: toutes les valeurs doivent être distinctes
        distinct_constraint = Distinct(self.cells)
        self.solver.add(distinct_constraint)
        
        # Contrainte de domaine: chaque valeur doit être entre 1 et 9
        for cell in self.cells:
            domain_constraint = And(cell >= 1, cell <= 9)
            self.solver.add(domain_constraint)
    
    def accepts(self, values: list) -> bool:
        """
        Vérifie si une séquence de valeurs est acceptée par l'automate.
        
        Args:
            values: Liste de 9 valeurs entières
            
        Returns:
            True si la séquence satisfait toutes les contraintes de ligne
        """
        if len(values) != 9:
            raise ValueError("Une ligne Sudoku doit contenir exactement 9 valeurs")
        
        # Créer un nouveau solver pour cette vérification
        s = Solver(ctx=self.ctx)
        
        # Ajouter les contraintes de l'automate
        s.add(Distinct(self.cells))
        for cell in self.cells:
            s.add(And(cell >= 1, cell <= 9))
        
        # Ajouter les contraintes spécifiques (les valeurs à tester)
        for cell, val in zip(self.cells, values):
            s.add(cell == val)
        
        # Vérifier la satisfiabilité
        return s.check() == sat
    
    def get_model(self) -> Optional[ModelRef]:
        """
        Retourne un modèle (une instance) de l'automate.
        
        Returns:
            Un modèle Z3 si satisfiable, None sinon
        """
        if self.solver.check() == sat:
            return self.solver.model()
        return None
    
    def __repr__(self) -> str:
        return f"RowAutomaton(row={self.row_idx})"

# Créer un automate pour la ligne 0
row_auto = RowAutomaton(0)
print(f"Automate créé: {row_auto}")

Automate créé: RowAutomaton(row=0)


#### Interprétation

La classe `RowAutomaton` est créée avec :

- **9 variables symboliques** `x_{row}_{j}` représentant chaque cellule de la ligne
- **Contrainte `Distinct`** : Garantit que toutes les valeurs sont différentes
- **Contraintes de domaine** : Chaque valeur est dans {1, ..., 9}

L'automate est maintenant prêt à vérifier des séquences de valeurs.

### 2.3 Tests de l'Automate de Ligne

Testons l'automate avec différentes séquences de valeurs.

In [3]:
# Test 1: Séquence valide (permutation de 1-9)
valid_sequence = [1, 2, 3, 4, 5, 6, 7, 8, 9]
result1 = row_auto.accepts(valid_sequence)
print(f"Test 1 - Séquence valide {valid_sequence}:")
print(f"  Acceptée? {result1}")
print(f"  Attendu: True (toutes les valeurs sont distinctes)")
print()

Test 1 - Séquence valide [1, 2, 3, 4, 5, 6, 7, 8, 9]:
  Acceptée? True
  Attendu: True (toutes les valeurs sont distinctes)



#### Interprétation

| Test | Séquence | Résultat | Attendu | Validation |
|------|----------|----------|----------|------------|
| 1 | [1,2,3,4,5,6,7,8,9] | True | True | Permutation valide |

La séquence contient tous les chiffres 1-9 exactement une fois, donc elle satisfait la contrainte de ligne. Z3 retourne `Sat` (satisfiable) car toutes les contraintes sont respectées.

In [4]:
# Test 2: Séquence invalide (doublon)
invalid_sequence = [1, 1, 3, 4, 5, 6, 7, 8, 9]
result2 = row_auto.accepts(invalid_sequence)
print(f"Test 2 - Séquence invalide {invalid_sequence}:")
print(f"  Acceptée? {result2}")
print(f"  Attendu: False (la valeur 1 apparaît deux fois)")
print()

Test 2 - Séquence invalide [1, 1, 3, 4, 5, 6, 7, 8, 9]:
  Acceptée? False
  Attendu: False (la valeur 1 apparaît deux fois)



#### Interprétation

| Test | Séquence | Résultat | Attendu | Validation |
|------|----------|----------|----------|------------|
| 2 | [1,1,3,4,5,6,7,8,9] | False | False | Doublon de 1 |

La contrainte `Distinct` est violée car la valeur 1 apparaît deux fois dans la ligne. Z3 retourne `Unsat` (unsatisfiable) lorsqu'il détecte cette violation.

In [5]:
# Test 3: Séquence avec valeur hors domaine
out_of_range = [1, 2, 3, 4, 5, 6, 7, 8, 10]
result3 = row_auto.accepts(out_of_range)
print(f"Test 3 - Séquence hors domaine {out_of_range}:")
print(f"  Acceptée? {result3}")
print(f"  Attendu: False (10 n'est pas dans {1, ..., 9})")

Test 3 - Séquence hors domaine [1, 2, 3, 4, 5, 6, 7, 8, 10]:
  Acceptée? False
  Attendu: False (10 n'est pas dans (1, Ellipsis, 9))


#### Interprétation

| Test | Séquence | Résultat | Attendu | Validation |
|------|----------|----------|----------|------------|
| 3 | [1,2,3,4,5,6,7,8,10] | False | False | Valeur 10 hors domaine |

La contrainte de domaine (valeurs entre 1 et 9) est violée. Z3 détecte cette inconsistance immédiatement lors de la vérification `s.check()`.

In [6]:
# Obtenir un modèle (une instance valide de la ligne)
model = row_auto.get_model()
if model:
    print("Une instance valide de la ligne 0:")
    values = [model.evaluate(row_auto.cells[j]).as_long() for j in range(9)]
    print(f"  {values}")
    print(f"  Vérification: {row_auto.accepts(values)}")
else:
    print("Aucun modèle disponible")

Une instance valide de la ligne 0:
  [1, 2, 3, 4, 5, 6, 7, 8, 9]
  Vérification: True


#### Interprétation

Z3 a généré automatiquement une instance valide de la ligne Sudoku. Cette instance satisfait toutes les contraintes :
- Toutes les valeurs sont distinctes
- Toutes les valeurs sont dans {1, ..., 9}

C'est la puissance des automates symboliques : Z3 peut générer des instances valides sans recherche explicite.

---

## 3. Automate Symbolique - Bloc 3x3 (15 min)

Après la ligne, modélisons maintenant la contrainte de **bloc 3x3** : toutes les valeurs dans un bloc doivent être distinctes.

In [7]:
class BlockAutomaton:
    """
    Automate symbolique pour la contrainte de bloc 3x3 Sudoku.
    
    Un bloc est identifié par sa position dans la grille 3x3 de blocs.
    Ex: le bloc (0,0) correspond aux cellules (0-2, 0-2).
    """
    
    def __init__(self, block_row: int, block_col: int):
        """
        Initialise l'automate pour un bloc spécifique.
        
        Args:
            block_row: Index du bloc en ligne (0-2)
            block_col: Index du bloc en colonne (0-2)
        """
        if not (0 <= block_row < 3 and 0 <= block_col < 3):
            raise ValueError("block_row et block_col doivent être dans {0, 1, 2}")
        
        self.block_row = block_row
        self.block_col = block_col
        self.ctx = Context()
        self.solver = SolverFor("QF_LIA", ctx=self.ctx)
        
        # Variables pour les 9 cellules du bloc
        self.cells = []
        for i in range(3):
            for j in range(3):
                # Convertir les coordonnées bloc en coordonnées grille
                row = 3 * block_row + i
                col = 3 * block_col + j
                self.cells.append(Int(f'x_{row}_{col}', ctx=self.ctx))
        
        # Contrainte: toutes les valeurs du bloc sont distinctes
        self.solver.add(Distinct(self.cells))
        
        # Contrainte de domaine
        for cell in self.cells:
            self.solver.add(And(cell >= 1, cell <= 9))
    
    def accepts(self, values: list) -> bool:
        """
        Vérifie si 9 valeurs forment un bloc valide.
        
        Args:
            values: Liste de 9 valeurs (ligne par ligne dans le bloc)
            
        Returns:
            True si le bloc est valide
        """
        if len(values) != 9:
            raise ValueError("Un bloc doit contenir exactement 9 valeurs")
        
        s = Solver(ctx=self.ctx)
        s.add(Distinct(self.cells))
        for cell in self.cells:
            s.add(And(cell >= 1, cell <= 9))
        
        for cell, val in zip(self.cells, values):
            s.add(cell == val)
        
        return s.check() == sat
    
    def get_cell_positions(self) -> list:
        """
        Retourne les positions des cellules du bloc dans la grille 9x9.
        
        Returns:
            Liste de tuples (row, col) pour les 9 cellules
        """
        positions = []
        for i in range(3):
            for j in range(3):
                positions.append((3 * self.block_row + i, 3 * self.block_col + j))
        return positions
    
    def __repr__(self) -> str:
        return f"BlockAutomaton(block=({self.block_row},{self.block_col}))"

# Créer un automate pour le bloc central (1, 1)
block_auto = BlockAutomaton(1, 1)
print(f"Automate créé: {block_auto}")
print(f"Cellules du bloc: {block_auto.get_cell_positions()}")

Automate créé: BlockAutomaton(block=(1,1))
Cellules du bloc: [(3, 3), (3, 4), (3, 5), (4, 3), (4, 4), (4, 5), (5, 3), (5, 4), (5, 5)]


#### Interprétation

L'automate de bloc (1,1) couvre les cellules centrales de la grille :

```
  0 1 2    3 4 5    6 7 8
0 . . .    . . .    . . .
1 . . .    X X X    . . .    <- Ligne 3-5
2 . . .    X X X    . . .

3 . . .    X X X    . . .
4 . . .    X X X    . . .    <- Colonne 3-5
5 . . .    X X X    . . .

6 . . .    . . .    . . .
7 . . .    . . .    . . .
8 . . .    . . .    . . .
```

Les coordonnées sont calculées automatiquement : `row = 3*block_row + i`, `col = 3*block_col + j`.

### 3.1 Tests de l'Automate de Bloc

In [8]:
# Test 1: Bloc valide (permutation)
valid_block = [5, 3, 4, 6, 7, 8, 9, 1, 2]
result1 = block_auto.accepts(valid_block)
print(f"Test 1 - Bloc valide {valid_block}:")
print(f"  Acceptée? {result1}")

Test 1 - Bloc valide [5, 3, 4, 6, 7, 8, 9, 1, 2]:
  Acceptée? True


#### Interprétation

Le bloc valide contient une permutation de {1, ..., 9}, satisfaisant la contrainte de distinctivité.

In [9]:
# Test 2: Bloc invalide (doublon)
invalid_block = [5, 3, 4, 6, 7, 8, 9, 1, 1]
result2 = block_auto.accepts(invalid_block)
print(f"Test 2 - Bloc invalide {invalid_block}:")
print(f"  Acceptée? {result2}")
print(f"  Attendu: False (la valeur 1 apparaît deux fois)")

Test 2 - Bloc invalide [5, 3, 4, 6, 7, 8, 9, 1, 1]:
  Acceptée? False
  Attendu: False (la valeur 1 apparaît deux fois)


---

## 4. Automate Complet Sudoku (25 min)

Maintenant, combinons tous les automates pour créer un automate complet qui modélise toutes les contraintes du Sudoku.

### 4.1 Architecture de l'Automate Complet

L'automate complet Sudoku combine :

1. **9 automates de ligne** - Une pour chaque ligne
2. **9 automates de colonne** - Une pour chaque colonne  
3. **9 automates de bloc** - Un pour chaque bloc 3x3

Ces automates partagent les mêmes variables symboliques `x_{i}_{j}`, créant ainsi un **réseau de contraintes interconnecté**.

In [10]:
class SudokuSymbolicAutomaton:
    """
    Automate symbolique complet pour Sudoku.
    
    Cet automate combine toutes les contraintes Sudoku (lignes, colonnes, blocs)
    et permet de résoudre des puzzles par recherche symbolique.
    """
    
    def __init__(self):
        """Initialise l'automate avec toutes les contraintes Sudoku."""
        self.ctx = Context()
        self.solver = SolverFor("QF_LIA", ctx=self.ctx)
        
        # Variables symboliques pour les 81 cellules
        # cells[i][j] représente la cellule à la ligne i, colonne j
        self.cells = [[Int(f'x_{i}_{j}', ctx=self.ctx) for j in range(9)] for i in range(9)]
        
        # Construire toutes les contraintes
        self._build_constraints()
    
    def _build_constraints(self):
        """Construit toutes les contraintes Sudoku."""
        self._build_row_constraints()
        self._build_column_constraints()
        self._build_block_constraints()
    
    def _build_row_constraints(self):
        """Ajoute les contraintes de ligne: valeurs distinctes dans chaque ligne."""
        for i in range(9):
            row = [self.cells[i][j] for j in range(9)]
            self.solver.add(Distinct(row))
            # Contraintes de domaine (une seule fois)
            for cell in row:
                self.solver.add(And(cell >= 1, cell <= 9))
    
    def _build_column_constraints(self):
        """Ajoute les contraintes de colonne: valeurs distinctes dans chaque colonne."""
        for j in range(9):
            col = [self.cells[i][j] for i in range(9)]
            self.solver.add(Distinct(col))
    
    def _build_block_constraints(self):
        """Ajoute les contraintes de bloc: valeurs distinctes dans chaque bloc 3x3."""
        for block_row in range(3):
            for block_col in range(3):
                block = []
                for i in range(3):
                    for j in range(3):
                        block.append(self.cells[3*block_row + i][3*block_col + j])
                self.solver.add(Distinct(block))
    
    def set_cell(self, row: int, col: int, value: int):
        """
        Ajoute une transition: fixe une cellule à une valeur donnée.
        
        Cette méthode ajoute une contrainte d'égalité, réduisant
        l'espace des états symboliques acceptables.
        
        Args:
            row: Ligne (0-8)
            col: Colonne (0-8)
            value: Valeur à assigner (1-9)
        """
        if not (0 <= row < 9 and 0 <= col < 9):
            raise ValueError("Position invalide")
        if not (1 <= value <= 9):
            raise ValueError("La valeur doit être entre 1 et 9")
        
        self.solver.add(self.cells[row][col] == value)
    
    def solve(self, timeout_ms: int = 5000) -> Optional[list]:
        """
        Cherche un état accepteur (une solution complète).
        
        Args:
            timeout_ms: Timeout en millisecondes
            
        Returns:
            Une grille 9x9 (liste de listes) si solution trouvée, None sinon
        """
        # Configurer le timeout
        self.solver.set(timeout=timeout_ms)
        
        # Chercher une solution
        status = self.solver.check()
        
        if status == sat:
            model = self.solver.model()
            # Extraire la solution
            solution = [
                [model.evaluate(self.cells[i][j]).as_long() for j in range(9)]
                for i in range(9)
            ]
            return solution
        elif status == unknown:
            print("Warning: Timeout ou indéterminé")
            return None
        else:
            print("Aucune solution trouvée (problème inconsistant)")
            return None
    
    def is_valid_placement(self, row: int, col: int, value: int) -> bool:
        """
        Vérifie si placer 'value' en (row, col) est valide.
        
        Args:
            row: Ligne (0-8)
            col: Colonne (0-8)
            value: Valeur à tester (1-9)
            
        Returns:
            True si le placement est valide
        """
        # Créer un solver temporaire pour tester
        test_solver = Solver(ctx=self.ctx)
        
        # Ajouter la contrainte à tester
        test_solver.add(self.cells[row][col] == value)
        
        # Copier les assertions du solver principal
        for assertion in self.solver.assertions():
            test_solver.add(assertion)
        
        return test_solver.check() == sat
    
    def count_solutions(self, max_solutions: int = 10) -> int:
        """
        Compte le nombre de solutions (jusqu'à max_solutions).
        
        Args:
            max_solutions: Nombre maximum de solutions à chercher
            
        Returns:
            Nombre de solutions trouvées
        """
        count = 0
        solutions = []
        
        for _ in range(max_solutions):
            result = self.solve()
            if result is None:
                break
            count += 1
            
            # Ajouter une contrainte pour éviter de retrouver la même solution
            block_constraint = Or([
                self.cells[i][j] != result[i][j]
                for i in range(9) for j in range(9)
            ])
            self.solver.add(block_constraint)
            solutions.append(result)
        
        return count
    
    def __repr__(self) -> str:
        return "SudokuSymbolicAutomaton()"

# Créer l'automate complet
automaton = SudokuSymbolicAutomaton()
print(f"Automate créé: {automaton}")

Automate créé: SudokuSymbolicAutomaton()


#### Interprétation

L'automate complet est initialisé avec :

- **81 variables symboliques** `x_{i}_{j}` pour chaque cellule
- **27 contraintes `Distinct`** : 9 lignes + 9 colonnes + 9 blocs
- **Contraintes de domaine** : Chaque variable dans {1, ..., 9}

Les méthodes principales :

| Méthode | Description |
|-----------|-------------|
| `set_cell(row, col, value)` | Ajoute une transition (fixe une cellule) |
| `solve()` | Cherche un état accepteur (solution complète) |
| `is_valid_placement(row, col, value)` | Teste une transition sans l'appliquer |
| `count_solutions()` | Compte le nombre de solutions |

L'automate est maintenant prêt à résoudre des puzzles Sudoku.

---

## 5. Exemples de Résolution (15 min)

Testons notre automate sur des puzzles Sudoku de difficultés variées.

### 5.1 Fonctions Utilitaires

D'abord, créons des fonctions pour afficher les grilles Sudoku de manière lisible.

In [11]:
def display_grid(grid: list):
    """
    Affiche une grille Sudoku de manière formatée.
    
    Args:
        grid: Liste de 9 listes de 9 entiers
    """
    for i in range(9):
        if i > 0 and i % 3 == 0:
            print("------+-------+------")
        row_str = ""
        for j in range(9):
            if j > 0 and j % 3 == 0:
                row_str += "| "
            val = grid[i][j] if grid[i][j] != 0 else '.'
            row_str += f"{val} "
        print(row_str)


def parse_grid(grid_str: str) -> list:
    """
    Convertit une chaîne de 81 caractères en grille 9x9.
    
    Args:
        grid_str: Chaîne de 81 caractères (0 ou . pour case vide)
        
    Returns:
        Liste de 9 listes de 9 entiers
    """
    grid_str = grid_str.replace('.', '0').replace(' ', '').replace('\n', '')
    if len(grid_str) != 81:
        raise ValueError(f"Attendu 81 caractères, reçu {len(grid_str)}")
    
    grid = []
    for i in range(9):
        row = [int(grid_str[i * 9 + j]) for j in range(9)]
        grid.append(row)
    return grid

def solve_puzzle(grid_str: str, timeout_ms: int = 5000) -> Optional[list]:
    """
    Résout un puzzle Sudoku avec l'automate symbolique.
    
    Args:
        grid_str: Chaîne représentant le puzzle
        timeout_ms: Timeout en millisecondes
        
    Returns:
        Grille solution ou None
    """
    automaton = SudokuSymbolicAutomaton()
    
    # Parser et appliquer les contraintes initiales
    grid = parse_grid(grid_str)
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                automaton.set_cell(i, j, grid[i][j])
    
    # Résoudre
    return automaton.solve(timeout_ms=timeout_ms)

print("Fonctions utilitaires définies.")

Fonctions utilitaires définies.


#### Interprétation

Les fonctions utilitaires sont prêtes :

- `display_grid(grid)` - Affiche une grille avec séparateurs de blocs
- `parse_grid(grid_str)` - Convertit une chaîne en grille 9x9
- `solve_puzzle(grid_str)` - Résout un puzzle complet

### 5.2 Sudoku Facile

Commençons par un puzzle facile avec beaucoup d'indices.

In [12]:
import time

# Puzzle facile (beaucoup d'indices)
EASY_PUZZLE = """
003020600
900305001
001806400
008102900
700000008
006708200
002609500
800203009
005010300
""".replace('\n', '').replace(' ', '')

print("Puzzle facile:")
initial_grid = parse_grid(EASY_PUZZLE)
display_grid(initial_grid)
print()

start = time.time()
solution = solve_puzzle(EASY_PUZZLE)
elapsed_ms = (time.time() - start) * 1000

if solution:
    print(f"Solution trouvée en {elapsed_ms:.2f} ms:")
    display_grid(solution)
else:
    print("Aucune solution trouvée")

Puzzle facile:
. . 3 | . 2 . | 6 . . 
9 . . | 3 . 5 | . . 1 
. . 1 | 8 . 6 | 4 . . 
------+-------+------
. . 8 | 1 . 2 | 9 . . 
7 . . | . . . | . . 8 
. . 6 | 7 . 8 | 2 . . 
------+-------+------
. . 2 | 6 . 9 | 5 . . 
8 . . | 2 . 3 | . . 9 
. . 5 | . 1 . | 3 . . 



Solution trouvée en 57.21 ms:
4 8 3 | 9 2 1 | 6 5 7 
9 6 7 | 3 4 5 | 8 2 1 
2 5 1 | 8 7 6 | 4 9 3 
------+-------+------
5 4 8 | 1 3 2 | 9 7 6 
7 2 9 | 5 6 4 | 1 3 8 
1 3 6 | 7 9 8 | 2 4 5 
------+-------+------
3 7 2 | 6 8 9 | 5 1 4 
8 1 4 | 2 5 3 | 7 6 9 
6 9 5 | 4 1 7 | 3 8 2 


#### Interprétation

Le puzzle facile a été résolu rapidement. Pour les puzzles faciles avec beaucoup d'indices, l'automate symbolique trouve très rapidement une solution car l'espace de recherche est fortement contraint dès le départ.

| Type de puzzle | Cases vides (typique) | Temps de résolution |
|----------------|----------------------|---------------------|
| Facile | 35-45 | Quelques millisecondes |
| Moyen | 45-55 | Quelques millisecondes |
| Difficile | 55-60 | Jusqu'à quelques secondes |

La performance de Z3 sur les Sudokus faciles est excellente car les contraintes initiales réduisent drastiquement l'espace de recherche.

### 5.3 Sudoku Difficile

Testons maintenant un puzzle difficile, présent dans le benchmark "top95".

In [13]:
# Puzzle difficile (moins d'indices)
HARD_PUZZLE = "4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......"

print("Puzzle difficile:")
initial_grid = parse_grid(HARD_PUZZLE)
display_grid(initial_grid)
print(f"Cases vides: {sum(1 for i in range(9) for j in range(9) if initial_grid[i][j] == 0)}")
print()

start = time.time()
solution = solve_puzzle(HARD_PUZZLE, timeout_ms=10000)
elapsed_ms = (time.time() - start) * 1000

if solution:
    print(f"Solution trouvée en {elapsed_ms:.2f} ms:")
    display_grid(solution)
else:
    print("Aucune solution trouvée (timeout ou inconsistant)")

Puzzle difficile:
4 . . | . . . | 8 . 5 
. 3 . | . . . | . . . 
. . . | 7 . . | . . . 
------+-------+------
. 2 . | . . . | . 6 . 
. . . | . 8 . | 4 . . 
. . . | . 1 . | . . . 
------+-------+------
. . . | 6 . 3 | . 7 . 
5 . . | 2 . . | . . . 
1 . 4 | . . . | . . . 
Cases vides: 64



Solution trouvée en 824.79 ms:
4 1 7 | 3 6 9 | 8 2 5 
6 3 2 | 1 5 8 | 9 4 7 
9 5 8 | 7 2 4 | 3 1 6 
------+-------+------
8 2 5 | 4 3 7 | 1 6 9 
7 9 1 | 5 8 6 | 4 3 2 
3 4 6 | 9 1 2 | 7 5 8 
------+-------+------
2 8 9 | 6 4 3 | 5 7 1 
5 7 3 | 2 9 1 | 6 8 4 
1 6 4 | 8 7 5 | 2 9 3 


#### Interprétation

Le puzzle difficile a été résolu avec succès. Les performances de l'automate symbolique dépendent de plusieurs facteurs :

1. **Nombre de cases vides** - Plus il y a de cases vides, plus l'espace de recherche est grand
2. **Emplacement des indices** - Certains placements contraignent plus fortement la grille
3. **Performance de Z3** - Le solveur SMT sous-jacent est très optimisé

Pour les puzzles difficiles, Z3 utilise des techniques avancées comme :
- La propagation de contraintes
- L'apprentissage de clauses (CDCL)
- La stratégie de décision heuristique

> **Note technique** : Les puzzles "top95" sont connus pour être difficiles pour les solveurs naïfs mais sont traités efficacement par Z3.

### 5.4 Test d'Unicité de la Solution

Un puzzle Sudoku bien formé doit avoir **exactement une solution**. Vérifions cela avec notre automate.

In [14]:
# Créer un nouvel automate pour tester l'unicité
automaton = SudokuSymbolicAutomaton()

# Appliquer les contraintes du puzzle facile
grid = parse_grid(EASY_PUZZLE)
for i in range(9):
    for j in range(9):
        if grid[i][j] != 0:
            automaton.set_cell(i, j, grid[i][j])

# Compter les solutions
print("Test d'unicité pour le puzzle facile...")
num_solutions = automaton.count_solutions(max_solutions=2)

if num_solutions == 1:
    print("Résultat: Le puzzle a exactement 1 solution (puzzle bien formé).")
elif num_solutions == 0:
    print("Résultat: Aucune solution (puzzle inconsistant).")
else:
    print(f"Résultat: Le puzzle a {num_solutions} solutions (puzzle mal formé).")

Test d'unicité pour le puzzle facile...
Aucune solution trouvée (problème inconsistant)
Résultat: Le puzzle a exactement 1 solution (puzzle bien formé).


#### Interprétation

L'unicité de la solution est une propriété importante pour les puzzles Sudoku :

| Nombre de solutions | Signification |
|---------------------|---------------|
| 1 | Puzzle bien formé (standard) |
| 0 | Puzzle inconsistant (contradictoire) |
| >1 | Puzzle mal formé (solutions multiples) |

L'automate symbolique permet de vérifier cette propriété formellement. La méthode `count_solutions` travaille en ajoutant des contraintes d'exclusion pour éviter de retrouver les mêmes solutions.

> **Note technique** : Pour vérifier l'unicité de façon optimale, il suffit de chercher une deuxième solution. Si aucune n'est trouvée, la première solution est unique.

---

## 6. Comparaison avec l'Approche CSP (10 min)

Comparons maintenant l'approche par automates symboliques avec l'approche CSP classique utilisant OR-Tools.

### 6.1 Tableau Comparatif

| Aspect | Automate Symbolique (Z3) | CSP (OR-Tools) | Commentaire |
|--------|-------------------------|----------------|-------------|
| **Modélisation** | Variables + contraintes logiques | Variables + domaines + contraintes | Les deux sont déclaratifs |
| **Résolution** | SMT solver (Z3) | CP-SAT solver | Z3 est plus général, OR-Tools plus spécialisé |
| **Recherche** | Interne (non visible) | Visible (branching) | OR-Tools permet plus de contrôle |
| **Extensibilité** | Très haute (formules arbitraires) | Moyenne (contraintes prédéfinies) | Z3 gère n'importe quelle formule |
| **Performance** | Bonne | Excellente | OR-Tools est optimisé pour les contraintes classiques |
| **Verbosité** | Compacte | Plus verbeuse | Z3 utilise des expressions plus succinctes |
| **Vérification** | Formelle (garantie) | Formelle (garantie) | Les deux garantissent la correction |

### 6.2 Avantages de l'Approche Symbolique

1. **Généralité** : Z3 peut gérer des contraintes arbitraires, pas seulement `AllDifferent`
2. **Preuve** : Z3 peut générer des preuves de satisfiabilité
3. **Théories** : Z3 supporte l'arithmétique, les tableaux, les bit-vectors, etc.
4. **Interactivité** : Facile à tester des hypothèses (méthode `is_valid_placement`)

### 6.3 Avantages de l'Approche CSP

1. **Performance** : OR-Tools est souvent plus rapide pour les contraintes classiques
2. **Propagation** : La propagation de contraintes est plus explicite
3. **Heuristiques** : Contrôle plus fin sur les heuristiques de recherche
4. **Industrialisation** : Plus mature pour les problèmes de production

---

## 7. Pourquoi Pas Automata.Net? (5 min)

Vous vous demandez peut-être pourquoi nous n'utilisons pas la bibliothèque **Automata.Net** pour les automates symboliques. Voici pourquoi :

### 7.1 Issue #6 - Bug Non Résolu

La bibliothèque Automata.Net a un bug critique non résolu (issue #6 sur GitHub) qui empêche son utilisation pour des cas réels.

### 7.2 Bibliothèque Obsolète

- Dernière mise à jour : 2017-2018
- Plus maintenue activement
- Problèmes de compatibilité avec .NET moderne

### 7.3 Notre Approche : Z3 Direct

Notre solution consiste à utiliser **Z3 directement** :

| Avantage | Description |
|----------|-------------|
| **Maintenance active** | Z3 est développé activement par Microsoft Research |
| **Stabilité** | Testé industriellement, utilisé dans de nombreux projets |
| **Performance** | Très optimisé, supporte de nombreuses théories |
| **Flexibilité** | Python, C#, C++, Java, JavaScript, etc. |
| **Automates symboliques** | Support natif via formules logiques |

Pour plus de détails sur les automates symboliques avec Z3, voir [Search-12-SymbolicAutomata](../Search/Foundations/Search-12-SymbolicAutomata.ipynb).

---

## 8. Conclusion

Dans ce notebook, nous avons exploré comment résoudre des puzzles Sudoku en utilisant des **automates symboliques** avec Z3. Voici un résumé des points clés :

### Résumé des Concepts

| Concept | Description |
|---------|-------------|
| **Automate symbolique** | Représentation logique d'un système de transitions |
| **État symbolique** | Formule représentant un ensemble d'états |
| **Transition** | Ajout d'une contrainte (assignation de valeur) |
| **État acceptant** | Solution satisfaisant toutes les contraintes |

### Points Clés Appris

1. **Modélisation** : Un Sudoku peut être modélisé comme un réseau d'automates symboliques (lignes, colonnes, blocs)
2. **Contraintes** : Z3 représente les contraintes de manière compacte avec `Distinct`
3. **Résolution** : Le solver Z3 trouve automatiquement un état accepteur
4. **Généralité** : L'approche s'étend à d'autres problèmes de contraintes

### Pour Aller Plus Loin

- **Sudoku 16x16** : Étendre l'automate aux grilles 4x4 de blocs 4x4
- **Contraintes additionnelles** : Ajouter des contraintes de diagonale (Sudoku X)
- **Génération** : Utiliser l'automate pour générer des puzzles uniques
- **Optimisation** : Comparer avec d'autres solveurs sur des benchmarks plus larges

## Exercices

### Exercice 1 : Validation d'Indices

Implémentez une méthode `validate_indices(grid_str)` qui vérifie si les indices donnés sont valides (ne violent pas les contraintes Sudoku).

### Exercice 2 : Comparaison de Performance

Comparez les temps de résolution entre l'automate symbolique et l'approche OR-Tools sur 10 puzzles de difficulté moyenne.

### Exercice 3 : Extension 16x16

Modifiez la classe `SudokuSymbolicAutomaton` pour supporter des grilles 16x16 (blocs 4x4, valeurs 1-16).

### Exercice 4 : Sudoku avec Diagonales

Ajoutez les contraintes de Sudoku X : les valeurs sur les deux diagonales principales doivent aussi être distinctes.

---

<< [Sudoku-11-Comparison](Sudoku-11-Comparison.ipynb) | [Index](README.md) >>

**Series connexes** :
- [Search-12-SymbolicAutomata](../Search/Foundations/Search-12-SymbolicAutomata.ipynb) - Théorie des automates symboliques
- [Sudoku-4-Z3](Sudoku-4-Z3.ipynb) - Z3 basics pour Sudoku
- [Sudoku-3-ORTools](Sudoku-3-ORTools.ipynb) - Approche CSP classique
- [SymbolicAI](../SymbolicAI/README.md) - Série complète sur l'IA symbolique